

ナーススケジュール問題を定義し，解きます．
シフト作成することで人件費の削減につながり，労働者の負担を少しでも減らすことができるのではないかと考え，行いました．

条件は以下の通りです．

*   看護師は"Taro","Sherry","Ai","Jack"の4人
*   Taroは4,Sherryは7,Aiは4,Jackは2の生産性を持つ
*   [Day1,Day2,Day3]の3日間のスケジュールを組む
*   Day1には6,Day2には7,Day3には6の生産性が必要
*   SherryはDay2できるだけお休み(ソフト制約)
*   AiはDay3できるだけお休み(ソフト制約)







In [43]:
!pip install qiskit
!pip install qiskit[optimization]
!pip install qiskit[visualization]

In [49]:
from qiskit_optimization import QuadraticProgram



In [50]:
# QuadraticProgramオブジェクトを生成します
problem = QuadraticProgram()

# 看護師とシフトのリストを定義します
nurses = ['Taro', 'Sherry', 'Ai','Jack']
days = ['Day1', 'Day2', 'Day3']  # 例えば3日分のスケジュールを考えます

# 看護師ごとの労働力を定義します
efficiency = {'Taro': 4, 'Sherry': 7, 'Ai': 4,'Jack':2}

# 日ごとの必要な労働力を定義します
workload = {'Day1': 6, 'Day2': 7, 'Day3': 6}



In [51]:
# バイナリ変数を追加します
for nurse in nurses:
        for day in days:
            problem.binary_var(f'{nurse}-{day}')


# 各日の必要な労働力が看護師の労働力の合計以上である制約を追加します
for day in days:
    problem.linear_constraint(linear={f'{nurse}-{day}': efficiency[nurse] for nurse in nurses}, sense='>=', rhs=workload[day])

penalty = 10  # 適切なペナルティ値を設定する

# 目的関数の辞書を初期化
objective = {i: 0 for i in range(len(problem.variables))}

# SherryはDay2休み
objective['Sherry-Day2'] = penalty

# AiはDay3休み
objective['Ai-Day3'] = penalty

# 目的関数を設定
problem.minimize(linear=objective)


print(problem.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 10 SherrymDay2 + 10 AimDay3
Subject To
 c0: 4 TaromDay1 + 7 SherrymDay1 + 4 AimDay1 + 2 JackmDay1 >= 6
 c1: 4 TaromDay2 + 7 SherrymDay2 + 4 AimDay2 + 2 JackmDay2 >= 7
 c2: 4 TaromDay3 + 7 SherrymDay3 + 4 AimDay3 + 2 JackmDay3 >= 6

Bounds
 0 <= TaromDay1 <= 1
 0 <= TaromDay2 <= 1
 0 <= TaromDay3 <= 1
 0 <= SherrymDay1 <= 1
 0 <= SherrymDay2 <= 1
 0 <= SherrymDay3 <= 1
 0 <= AimDay1 <= 1
 0 <= AimDay2 <= 1
 0 <= AimDay3 <= 1
 0 <= JackmDay1 <= 1
 0 <= JackmDay2 <= 1
 0 <= JackmDay3 <= 1

Binaries
 TaromDay1 TaromDay2 TaromDay3 SherrymDay1 SherrymDay2 SherrymDay3 AimDay1
 AimDay2 AimDay3 JackmDay1 JackmDay2 JackmDay3
End



In [52]:
from qiskit import Aer
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.utils import QuantumInstance

# QuantumInstanceを作成します
quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'))

# QAOAを使った最適化ソルバーを作成します
qaoa = QAOA(optimizer=COBYLA(maxiter=500), quantum_instance=quantum_instance)
qaoa_solver = MinimumEigenOptimizer(qaoa)

# 問題をQUBO形式に変換します
converter = QuadraticProgramToQubo()
qubo_problem = converter.convert(problem)

# QAOAソルバーで問題を解きます
qaoa_result = qaoa_solver.solve(qubo_problem)


<ipython-input-52-0d563be7ece9>:9: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'))
<ipython-input-52-0d563be7ece9>:12: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(optimizer=COBYLA(maxiter=500), quantum_instance=quantum_instance)


In [53]:
print("QAOA solution:", qaoa_result)

QAOA solution: fval=0.0, Taro-Day1=0.0, Taro-Day2=1.0, Taro-Day3=1.0, Sherry-Day1=1.0, Sherry-Day2=0.0, Sherry-Day3=0.0, Ai-Day1=1.0, Ai-Day2=1.0, Ai-Day3=0.0, Jack-Day1=0.0, Jack-Day2=1.0, Jack-Day3=1.0, c0@int_slack@0=1.0, c0@int_slack@1=0.0, c0@int_slack@2=0.0, c0@int_slack@3=1.0, c1@int_slack@0=0.0, c1@int_slack@1=0.0, c1@int_slack@2=0.0, c1@int_slack@3=1.0, c2@int_slack@0=0.0, c2@int_slack@1=0.0, c2@int_slack@2=0.0, c2@int_slack@3=0.0, status=SUCCESS
